In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gurobipy import Model, GRB, quicksum

In [2]:
gal_points = pd.read_csv("../archive/2023-fall-clinic/data/indoor_outdoor_pts_galv.csv")

In [3]:
gal_points.head(20)

,Name,Longitude,Latitude,Daily_Pickup_Totes,Weekly_Dropoff_Totes,user_ratings_total,category,rating,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,location_type,pickup_type,id
0,Moody Gardens,-94.852300,29.273600,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,depot,Truck,NaN
1,Sugar Bean,-94.870418,29.261029,1,1,476.0,cafe,4.7,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,indoor,Truck,NaN
2,MAC-IES,-94.791385,29.298912,1,1,73.0,restaurant,5.0,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,indoor,Truck,NaN
3,Good Dough,-94.808531,29.290612,1,1,68.0,bakery,4.6,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,indoor,Truck,NaN
4,Porch Cafe,-94.750192,29.319581,1,1,332.0,restaurant,4.1,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,indoor,Truck,NaN
5,Coastal Grill,-94.789455,29.308158,1,1,814.0,restaurant,4.6,500,100,500,"1827 The Strand, Galveston",POINT (-94.789455 29.308158),Skeptical,High,No,Med,indoor,Truck,NaN
6,Texas A&M University at Galveston,-94.818287,29.315497,1,1,121.0,university,4.6,?,NaN,?,"200 Seawolf Parkway, Galveston",POINT (-94.8182868 29.3154967),Curious,High,No,High,indoor,Truck,NaN
7,Rudy & Paco Restaurant and Bar,-94.791081,29.305336,1,1,892.0,bar,4.7,100,75,100,"2028 Postoffice St, Galveston",POINT (-94.791081 29.3053363),Hesitant,High,No,High,indoor,Truck,NaN
8,Vargas Cut and Catch (listed above),-94.791385,29.305227,1,1,297.0,bar,4.8,35,49,35,"2102 Postoffice Street, Galveston",POINT (-94.7913854 29.3052268),Curious,High,No,High,indoor,Truck,NaN
9,MarMo Cafe & Lounge,-94.792103,29.305600,1,1,19.0,bar,4.8,140,35,140,"2121 Market Street suite 101, Galveston",POINT (-94.792103 29.3056002),High,High,No,Med,indoor,Truck,NaN


In [4]:
gal_dis = pd.read_csv("../archive/2023-fall-clinic/data/indoor_outdoor_distances_galv.csv")
gal_dis

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.0,5143.2,8990.2,6368.1,13815.7,10253.6,9602.4,9632.7,9629.0,9691.5,...,5572.7,7771.3,1849.6,1672.0,13248.8,6031.2,8261.5,11893.4,15070.8,14999.4
1,5286.6,0.0,12326.0,9623.2,15375.9,13589.5,12938.2,12968.5,12964.8,13027.3,...,3351.3,10925.6,5359.4,3439.3,16584.6,3763.6,6040.2,9672.0,12849.4,12778.1
2,8678.1,11909.4,0.0,2614.5,5088.0,1389.6,7453.6,905.0,901.3,967.6,...,11232.0,1136.5,7011.3,8690.0,11100.0,11690.5,13920.8,17552.7,20730.0,20658.7
3,6410.6,9642.0,2574.4,0.0,7399.9,3701.5,5968.7,3216.9,3213.2,3275.7,...,9453.6,2756.7,4743.8,6422.5,9615.2,9912.2,12142.5,15774.3,18951.7,18880.4
4,14122.0,17353.3,5111.7,7424.9,0.0,5271.7,11966.6,5250.3,5246.6,5542.6,...,15424.6,6248.1,12243.7,14133.9,15613.0,15883.2,18113.5,21745.3,24922.7,24851.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,6173.0,3785.7,11718.9,9889.1,15826.3,14392.5,13741.2,12845.9,12842.2,12904.8,...,1109.9,10974.9,6245.8,4354.0,17387.6,0.0,2555.5,6187.4,9364.8,9293.4
252,8419.0,6074.9,13964.8,12135.1,18072.3,16638.5,15987.2,15091.9,15088.2,15150.8,...,3355.9,13220.9,8491.8,6600.0,19633.6,2513.0,0.0,3966.8,7144.2,7072.9
253,12035.0,9690.9,17580.9,15751.1,21688.3,20254.5,19603.2,18707.9,18704.3,18766.8,...,6971.9,16836.9,12107.9,10216.0,23249.6,6129.0,3893.7,0.0,3615.7,3544.3
254,15209.0,12864.9,20754.8,18925.1,24862.3,23428.5,22777.2,21881.9,21878.2,21940.8,...,10145.9,20010.9,15281.8,13390.0,26423.6,9303.0,7067.6,3434.3,0.0,172.2


In [5]:
# number of location includes depot
n = gal_points.shape[0]
n

256

In [6]:
# N is the list of points (pickup and drop off)
N = [i for i in range(1, n)]
# V all the vertices including depot
V = [0] + N

In [7]:
V

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [8]:
# arc
A = [(i, j) for i in V for j in V if i != j]
A

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 39),
 (0, 40),
 (0, 41),
 (0, 42),
 (0, 43),
 (0, 44),
 (0, 45),
 (0, 46),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 51),
 (0, 52),
 (0, 53),
 (0, 54),
 (0, 55),
 (0, 56),
 (0, 57),
 (0, 58),
 (0, 59),
 (0, 60),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 64),
 (0, 65),
 (0, 66),
 (0, 67),
 (0, 68),
 (0, 69),
 (0, 70),
 (0, 71),
 (0, 72),
 (0, 73),
 (0, 74),
 (0, 75),
 (0, 76),
 (0, 77),
 (0, 78),
 (0, 79),
 (0, 80),
 (0, 81),
 (0, 82),
 (0, 83),
 (0, 84),
 (0, 85),
 (0, 86),
 (0, 87),
 (0, 88),
 (0, 89),
 (0, 90),
 (0, 91),
 (0, 92),
 (0, 93),
 (0, 94),
 (0, 95),
 (0, 96),
 (0, 97),
 (0, 98),
 (0, 99),
 (0, 100),
 (0, 101

In [9]:
# access the distance
gal_dis.iloc[0,3]

6368.1

In [10]:
# create dictionary for each arc for each pair of vertices
c = {(i, j): gal_dis.iloc[i,j] for i, j in A}

In [11]:
c

{(0, 1): 5143.2,
 (0, 2): 8990.2,
 (0, 3): 6368.1,
 (0, 4): 13815.7,
 (0, 5): 10253.6,
 (0, 6): 9602.4,
 (0, 7): 9632.7,
 (0, 8): 9629.0,
 (0, 9): 9691.5,
 (0, 10): 9789.6,
 (0, 11): 9912.8,
 (0, 12): 9626.1,
 (0, 13): 9141.5,
 (0, 14): 9323.5,
 (0, 15): 6932.9,
 (0, 16): 5437.0,
 (0, 17): 6324.5,
 (0, 18): 7324.4,
 (0, 19): 5784.9,
 (0, 20): 8990.2,
 (0, 21): 9851.1,
 (0, 22): 2976.7,
 (0, 23): 9320.9,
 (0, 24): 9348.9,
 (0, 25): 5812.3,
 (0, 26): 9218.5,
 (0, 27): 9356.6,
 (0, 28): 10398.4,
 (0, 29): 10244.5,
 (0, 30): 10239.8,
 (0, 31): 2489.4,
 (0, 32): 8867.1,
 (0, 33): 5797.3,
 (0, 34): 10152.0,
 (0, 35): 2989.0,
 (0, 36): 10066.4,
 (0, 37): 9109.1,
 (0, 38): 9129.5,
 (0, 39): 5285.3,
 (0, 40): 8752.5,
 (0, 41): 6361.5,
 (0, 42): 7657.9,
 (0, 43): 8490.2,
 (0, 44): 6492.2,
 (0, 45): 18621.5,
 (0, 46): 8033.0,
 (0, 47): 10125.1,
 (0, 48): 9665.0,
 (0, 49): 10155.4,
 (0, 50): 10077.6,
 (0, 51): 8182.2,
 (0, 52): 8256.1,
 (0, 53): 9946.0,
 (0, 54): 9588.7,
 (0, 55): 9584.8,
 (0, 56)

In [12]:
# Q: number of vehicle
Q = 20
# q pickup/dropof capacity for each vertix
capacity_pickup = gal_points['Daily_Pickup_Totes']
q = {i: capacity_pickup.iloc[i] for i in N}
q

{1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 3,
 18: 1,
 19: 1,
 20: 1,
 21: 2,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 2,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 2,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 2,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 2,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 2,
 55: 1,
 56: 1,
 57: 8,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 2,
 64: 1,
 65: 1,
 66: 2,
 67: 1,
 68: 2,
 69: 1,
 70: 1,
 71: 1,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 2,
 77: 1,
 78: 2,
 79: 1,
 80: 1,
 81: 2,
 82: 2,
 83: 1,
 84: 1,
 85: 2,
 86: 1,
 87: 1,
 88: 1,
 89: 1,
 90: 2,
 91: 2,
 92: 1,
 93: 1,
 94: 1,
 95: 1,
 96: 1,
 97: 2,
 98: 2,
 99: 1,
 100: 2,
 101: 1,
 102: 1,
 103: 1,
 104: 2,
 105: 1,
 106: 2,
 107: 1,
 108: 1,
 109: 1,
 110: 2,
 111: 11,
 112: 1,
 113: 1,
 114: 1,
 115: 1,
 116: 1,
 117: 3,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 1,
 123: 1,


In [16]:
mdl = Model('CVRP')
# adding variable of location index
x = mdl.addVars(A, vtype=GRB.BINARY)
# adding variable of number of client to the model
u = mdl.addVars(N, vtype=GRB.CONTINUOUS)
# setup the goal of this model going to achieve
# minimize the total distance
mdl.modelSense = GRB.MINIMIZE
mdl.setObjective(quicksum(x[i,j]*c[i,j] for i,j in A))
mdl.addConstrs(quicksum(x[i, j] for j in V if j != i) == 1 for i in N)
mdl.addConstrs(quicksum(x[i, j] for i in V if i != j) == 1 for j in N)
mdl.addConstrs((x[i, j] == 1) >> (u[i]+q[j] == u[j])
               for i, j in A if i != 0 and j != 0)
# mdl.addConstrs(u[i] >= q[i] for i in N)
# might work this way for both pickup and drop off, add constraints that each points's load can not be negative
mdl.addConstrs(u[i] + q[i] >= 0 for i in N)
mdl.addConstrs(u[i] <= Q for i in N)
mdl.Params.MIPGap = 0.1
mdl.Params.TimeLimit = 100  # seconds
mdl.optimize()

Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 100
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 21.5.0 21F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1020 rows, 65535 columns and 130560 nonzeros
Model fingerprint: 0xf61bec9a
Model has 64770 general constraints
Variable types: 255 continuous, 65280 integer (65280 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve added 129030 rows and 0 columns
Presolve time: 1.39s
Presolved: 130050 rows, 65535 columns, 518670 nonzeros
Variable types: 255 continuous, 65280 integer (65280 binary)
Found heuristic solution: objective 4184511.7000

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier 

In [18]:
active_arcs = [a for a in A if x[a].x > 0.999]
selected_arcs = active_arcs.copy()
selected_arcs

[(0, 27),
 (0, 35),
 (0, 68),
 (0, 73),
 (0, 92),
 (0, 118),
 (0, 139),
 (0, 140),
 (0, 156),
 (0, 160),
 (0, 166),
 (0, 167),
 (0, 191),
 (0, 203),
 (0, 212),
 (0, 221),
 (0, 241),
 (1, 214),
 (2, 20),
 (3, 41),
 (4, 197),
 (5, 58),
 (6, 168),
 (7, 48),
 (8, 105),
 (9, 16),
 (10, 11),
 (11, 62),
 (12, 183),
 (13, 110),
 (14, 23),
 (15, 235),
 (16, 201),
 (17, 18),
 (18, 63),
 (19, 99),
 (20, 247),
 (21, 0),
 (22, 181),
 (23, 0),
 (24, 77),
 (25, 96),
 (26, 69),
 (27, 165),
 (28, 144),
 (29, 28),
 (30, 194),
 (31, 249),
 (32, 38),
 (33, 251),
 (34, 49),
 (35, 22),
 (36, 59),
 (37, 229),
 (38, 198),
 (39, 244),
 (40, 37),
 (41, 19),
 (42, 3),
 (43, 238),
 (44, 94),
 (45, 254),
 (46, 84),
 (47, 53),
 (48, 101),
 (49, 186),
 (50, 148),
 (51, 52),
 (52, 46),
 (53, 61),
 (54, 180),
 (55, 54),
 (56, 55),
 (57, 206),
 (58, 147),
 (59, 150),
 (60, 151),
 (61, 184),
 (62, 173),
 (63, 250),
 (64, 153),
 (65, 64),
 (66, 65),
 (67, 154),
 (68, 10),
 (69, 172),
 (70, 155),
 (71, 179),
 (72, 0),
 (7

In [21]:
selected_arcs[0]
pairs_starting_with_0 = [(i, j) for i, j in selected_arcs if i == 0]
pairs_starting_with_0

[(0, 27),
 (0, 35),
 (0, 68),
 (0, 73),
 (0, 92),
 (0, 118),
 (0, 139),
 (0, 140),
 (0, 156),
 (0, 160),
 (0, 166),
 (0, 167),
 (0, 191),
 (0, 203),
 (0, 212),
 (0, 221),
 (0, 241)]

In [26]:
def trace_route(route, start, selected_arcs):
    route.append(start)
    while True:
        found_next = False
        for arc in selected_arcs:
            if arc[0] == route[-1]:
                route.append(arc[1])
                selected_arcs.remove(arc)
                found_next = True
                break
        if not found_next:
            break
    return route

pairs_starting_with_0 = [(i, j) for i, j in selected_arcs if i == 0]
for start in pairs_starting_with_0:
    active_arcs.remove(start)

route_num = 1
for start in pairs_starting_with_0:
    route = [0]
    route = trace_route(route, start[1], list(active_arcs))  # Creating a copy of selected_arcs as it will be modified
    print(f"Route{route_num}:", " -> ".join(map(str, route)))
    route_num += 1

Route1: 0 -> 27 -> 165 -> 13 -> 110 -> 26 -> 69 -> 172 -> 12 -> 183 -> 67 -> 154 -> 70 -> 155 -> 24 -> 77 -> 14 -> 23 -> 0
Route2: 0 -> 35 -> 22 -> 181 -> 190 -> 189 -> 209 -> 207 -> 108 -> 162 -> 97 -> 42 -> 3 -> 41 -> 19 -> 99 -> 202 -> 0
Route3: 0 -> 68 -> 10 -> 11 -> 62 -> 173 -> 176 -> 175 -> 174 -> 188 -> 123 -> 17 -> 18 -> 63 -> 250 -> 6 -> 168 -> 0
Route4: 0 -> 73 -> 75 -> 236 -> 228 -> 164 -> 29 -> 28 -> 144 -> 103 -> 119 -> 74 -> 43 -> 238 -> 40 -> 37 -> 229 -> 30 -> 194 -> 182 -> 21 -> 0
Route5: 0 -> 92 -> 93 -> 232 -> 95 -> 25 -> 96 -> 237 -> 51 -> 52 -> 46 -> 84 -> 83 -> 88 -> 86 -> 158 -> 85 -> 87 -> 233 -> 91 -> 0
Route6: 0 -> 118 -> 126 -> 124 -> 122 -> 127 -> 246 -> 216 -> 170 -> 159 -> 1 -> 214 -> 215 -> 128 -> 39 -> 244 -> 0
Route7: 0 -> 139 -> 125 -> 141 -> 0
Route8: 0 -> 140 -> 220 -> 131 -> 60 -> 151 -> 219 -> 45 -> 254 -> 217 -> 145 -> 143 -> 135 -> 218 -> 253 -> 255 -> 33 -> 251 -> 252 -> 0
Route9: 0 -> 156 -> 4 -> 197 -> 171 -> 109 -> 177 -> 120 -> 47 -> 53 -> 